In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
train_var = pd.read_csv('/Users/senchen/Desktop/ds/training_variants')
test_var = pd.read_csv('/Users/senchen/Desktop/ds/test_variants')

train_text = pd.read_csv('/Users/senchen/Desktop/ds/training_text',sep='\|\|',
                    skiprows=1,engine='python',names=["ID","text"],encoding="utf-8")
test_text = pd.read_csv('/Users/senchen/Desktop/ds/test_text',sep='\|\|',
                    skiprows=1,engine='python',names=["ID","text"])

In [2]:
ID_train = train_var.ID
ID_test = test_var.ID
y = train_var.Class.values -1

In [4]:
stop_words = set(stopwords.words("english"))
def clean_text(txt):
    txt = re.sub("[^A-Za-z0-9^,!.\/'+-=]", " ", txt)
    txt =txt.lower().split()
    txt = [s for s in txt if not s in stop_words]
    txt = ' '.join(txt)
    txt =txt.replace(','," ")
    txt = txt.replace('.'," ")
    return(txt)

In [5]:
tra_text = []#list
tes_text = []#list
for item in train_text['text']:
    tra_text.append(clean_text(item))
for item in test_text['text']:
    tes_text.append(clean_text(item)) 
    
All_text = pd.concat((train_text,test_text), axis=0,ignore_index=True)

In [6]:
def gene_position(gene, txt,pos):
    i = pos
    while i != (len(txt)-1):
        if (txt[i] == gene):
            return i
        else:
            i += 1
    return "Not Found"

In [7]:
def get_nei(pos,n,txt):
    result = []
    if (pos-n) >= 0 & (pos+n) <= len(txt) +1:
        left = pos-n
        right = pos+n+1
        result.append(txt[left:right])
    elif (pos-n <0) & (pos + n) <= len(txt)+1:
        left = 0
        right = pos + n+1
        result.append(txt[left:right])
    elif (pos + n) > (len(txt)+1)  & (pos -n >=0):
        left = pos-n
        right = len(txt) +1
        result.append(txt[left:right])
    elif (pos +n) >(len(txt)+1) & (pos - n <= 0):
        left =0
        right = len(txt)
        result.append(txt[left:right])
    return result

In [8]:
def build_new_text(gene,txt,n):
    if gene_position(gene,txt,0) == "Not Found":
        gene= "mutation"
    pos = 0
    result=""
    while pos <= len(txt):
        pos = gene_position(gene,txt,pos)
        if (pos != "Not Found"):
            result = result + ' '.join(get_nei(pos,n,txt)[0])
            pos += 1
        elif (pos == "Not Found"):
            break
    return result

In [9]:
def build_mutation_text(gene,txt,n):
    if gene_position(gene,txt,0) == "Not Found":
        return ''
    pos = 0
    result=""
    while pos <= len(txt):
        pos = gene_position(gene,txt,pos)
        if (pos != "Not Found"):
            result = result + ' '.join(get_nei(pos,n,txt)[0])
            pos += 1
        elif (pos == "Not Found"):
            break
    return result

In [10]:
new_tra_fea =[]
new_tes_fea =[]
for i in range(3321):
    if build_new_text(train_var['Gene'][i].lower(),tra_text[i].split(),30) != '':
        new_tra_fea.append(build_new_text(train_var['Gene'][i].lower(),tra_text[i].split(),30))
    else:
        new_tra_fea.append(tra_text[i])
        
for i in range(5668):
    if build_new_text(test_var['Gene'][i].lower(),tes_text[i].split(),30) != '':
        new_tes_fea.append(build_new_text(test_var['Gene'][i].lower(),tes_text[i].split(),30))
    else:
        new_tes_fea.append(tes_text[i])

In [13]:
tra_vari_text =[]
tes_vari_text =[]
for i in range(3321):
    if build_new_text(train_var['Variation'][i].lower(),tra_text[i].split(),30) != '':
        tra_vari_text.append(build_new_text(train_var['Variation'][i].lower(),tra_text[i].split(),30))
    else:
        tra_vari_text.append(tra_text[i])
        #tra_vari_text.append("")
for i in range(5668):
    if build_new_text(test_var['Variation'][i].lower(),tes_text[i].split(),30) != '':
        tes_vari_text.append(build_new_text(test_var['Variation'][i].lower(),tes_text[i].split(),30))
    else:
        tes_vari_text.append(tes_text[i])
        #tes_vari_text.append("")

In [14]:
tra_muts_text =[]
tes_muts_text =[]
for i in range(3321):
    if build_mutation_text("mutations",tra_text[i].split(),30) != '':
        tra_muts_text.append(build_mutation_text("mutations",tra_text[i].split(),30))
    else:
        tra_muts_text.append("")
        
for i in range(5668):
    if build_mutation_text("mutations",tes_text[i].split(),30) != '':
        tes_muts_text.append(build_mutation_text("mutations",tes_text[i].split(),30))
    else:
        tes_muts_text.append("")

In [15]:
tra_cells_text =[]
tes_cells_text =[]
for i in range(3321):
    if build_mutation_text("cells",tra_text[i].split(),30) != '':
        tra_cells_text.append(build_mutation_text("cells",tra_text[i].split(),30))
    else:
        tra_cells_text.append("")
        
for i in range(5668):
    if build_mutation_text("cells",tes_text[i].split(),30) != '':
        tes_cells_text.append(build_mutation_text("cells",tes_text[i].split(),30))
    else:
        tes_cells_text.append("")

In [16]:
tra_cell_text =[]
tes_cell_text =[]
for i in range(3321):
    if build_mutation_text("cell",tra_text[i].split(),30) != '':
        tra_cell_text.append(build_mutation_text("cell",tra_text[i].split(),30))
    else:
        tra_cell_text.append("")
        
for i in range(5668):
    if build_mutation_text("cell",tes_text[i].split(),30) != '':
        tes_cell_text.append(build_mutation_text("cell",tes_text[i].split(),30))
    else:
        tes_cell_text.append("")

In [17]:
tra_can_text =[]
tes_can_text =[]
for i in range(3321):
    if build_mutation_text("cancer",tra_text[i].split(),30) != '':
        tra_can_text.append(build_mutation_text("cancer",tra_text[i].split(),30))
    else:
        tra_can_text.append("")
        
for i in range(5668):
    if build_mutation_text("cancer",tes_text[i].split(),30) != '':
        tes_can_text.append(build_mutation_text("cancer",tes_text[i].split(),30))
    else:
        tes_can_text.append("")

In [18]:
tra_x_text =[]
tes_x_text =[]
for i in range(3321):
    if build_mutation_text("cancer",tra_text[i].split(),30) != '':
        tra_can_text.append(build_mutation_text("cancer",tra_text[i].split(),30))
    else:
        tra_can_text.append("")
        
for i in range(5668):
    if build_mutation_text("cancer",tes_text[i].split(),30) != '':
        tes_can_text.append(build_mutation_text("cancer",tes_text[i].split(),30))
    else:
        tes_can_text.append("")

In [19]:
def count_word(var, words):
    if var.lower() in words:
        return 1
    else:
        return 0

In [20]:
train_count_gene = []
train_count_vari =[]
for i in range(len(train_var)):
    train_count_gene.append(count_word(train_var['Gene'][i],tra_text[i]))

for i in range(len(train_var)):
    train_count_vari.append(count_word(train_var['Variation'][i],tra_text[i]))

In [21]:
test_count_gene = []
test_count_vari =[]
for i in range(len(test_var)):
    test_count_gene.append(count_word(test_var['Gene'][i],tes_text[i]))

for i in range(len(test_var)):
    test_count_vari.append(count_word(test_var['Variation'][i],tes_text[i]))

In [22]:
train_clean_text = {'text' : tra_text}
df_train_clean_text = pd.DataFrame.from_dict(train_clean_text)
test_clean_text = {'text' : tes_text}
df_test_clean_text = pd.DataFrame.from_dict(test_clean_text)

In [23]:
df_train_new_text = pd.DataFrame(data=new_tra_fea,columns=['text'])
df_test_new_text = pd.DataFrame(data=new_tes_fea,columns=['text'])

In [24]:
df_train_vari_text = pd.DataFrame(data=tra_vari_text,columns=['text'])
df_test_vari_text = pd.DataFrame(data=tes_vari_text,columns=['text'])

In [26]:
df_train_muts_text = pd.DataFrame(data=tra_muts_text,columns=['text'])
df_test_muts_text = pd.DataFrame(data=tes_muts_text,columns=['text'])

In [27]:
df_train_cells_text = pd.DataFrame(data=tra_cells_text,columns=['text'])
df_test_cells_text = pd.DataFrame(data=tes_cells_text,columns=['text'])

In [28]:
df_train_cell_text = pd.DataFrame(data=tra_cell_text,columns=['text'])
df_test_cell_text = pd.DataFrame(data=tes_cell_text,columns=['text'])

In [29]:
df_train_can_text = pd.DataFrame(data=tra_can_text,columns=['text'])
df_test_can_text = pd.DataFrame(data=tes_can_text,columns=['text'])

In [30]:
df_train_x_text = pd.DataFrame(data=tra_x_text,columns=['text'])
df_test_x_text = pd.DataFrame(data=tes_x_text,columns=['text'])

In [31]:
TT_var = pd.concat((train_var.drop(['ID','Class'],axis=1),test_var.drop(['ID'],axis=1)), axis=0,ignore_index=True)

In [32]:
TT_text = pd.concat((df_train_clean_text,df_test_clean_text),axis=0,ignore_index=True)

In [33]:
TT_new_text = pd.concat((df_train_new_text,df_test_new_text),axis=0,ignore_index=True)

In [34]:
TT_vari_text = pd.concat((df_train_vari_text,df_test_vari_text),axis=0,ignore_index=True)

In [36]:
TT_muts_text = pd.concat((df_train_muts_text,df_test_muts_text),axis=0,ignore_index=True)

In [37]:
TT_cells_text = pd.concat((df_train_cells_text,df_test_cells_text),axis=0,ignore_index=True)

In [38]:
TT_cell_text = pd.concat((df_train_cell_text,df_test_cell_text),axis=0,ignore_index=True)

In [39]:
TT_can_text = pd.concat((df_train_can_text,df_test_can_text),axis=0,ignore_index=True)

In [40]:
TT_x_text = pd.concat((df_train_x_text,df_test_x_text),axis=0,ignore_index=True)

In [43]:
tfidf1 = TfidfVectorizer(min_df=5, ngram_range=(1,3), max_features=1000,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english').fit(TT_new_text['text'])

In [44]:
tfidf5= TfidfVectorizer(min_df=5, ngram_range=(1,3), max_features=1000,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english').fit(TT_vari_text['text'])

In [46]:
tfidf7= TfidfVectorizer(min_df=5, ngram_range=(1,3), max_features=1000,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english').fit(TT_muts_text['text'])

In [47]:
tfidf8= TfidfVectorizer(min_df=5, ngram_range=(1,3), max_features=1000,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english').fit(TT_cells_text['text'])

In [48]:
tfidf9= TfidfVectorizer(min_df=5, ngram_range=(1,3), max_features=1000,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english').fit(TT_cell_text['text'])

In [49]:
tfidf10= TfidfVectorizer(min_df=5, ngram_range=(1,3), max_features=1000,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english').fit(TT_can_text['text'])

In [51]:
from sklearn.preprocessing import normalize, Imputer
from sklearn.preprocessing import LabelEncoder
import scipy.sparse as ssp
import math
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing
def get_features(vari,text,tx):
    temp = vari.copy()
    lbl = LabelEncoder()
    temp['Gene_len'] = temp['Gene'].map(lambda x : len(x))
    temp['Gene_lbl_cnc'] = lbl.fit_transform(temp['Gene'].values)
    temp['Var_len'] = temp['Variation'].map(lambda x : len(x))
    temp['Var_lbl_cnc'] = lbl.fit_transform(temp['Variation'].values)
    temp['text_len'] =  tx['text'].map(lambda x : len(x))
    temp['text_word_count'] =  tx['text'].map(lambda x : len(str(x).split(' ')))
    return temp

In [52]:
TT_data = get_features(TT_var,TT_new_text['text'],All_text)

In [53]:
TT_data = TT_data.drop(['Gene'],axis=1)
TT_data = TT_data.drop(['Variation'],axis=1)

In [54]:
TT_data.head()

,Gene_len,Gene_lbl_cnc,Var_len,Var_lbl_cnc,text_len,text_word_count
0,6,447,20,7654,39672,6105
1,3,216,5,8255,36691,5783
2,3,216,5,5191,36691,5783
3,3,216,5,4572,36238,5625
4,3,216,5,3958,41308,6248


In [55]:
new_text_tfidf = tfidf1.transform(TT_new_text['text']) #reduced text gene

In [56]:
vari_text_tfidf = tfidf5.transform(TT_vari_text['text'])

In [58]:
muts_text_tfidf = tfidf7.transform(TT_muts_text['text'])

In [59]:
cells_text_tfidf = tfidf8.transform(TT_cells_text['text'])

In [60]:
cell_text_tfidf = tfidf9.transform(TT_cells_text['text'])

In [61]:
can_text_tfidf = tfidf10.transform(TT_can_text['text'])

In [62]:
tsvd = TruncatedSVD(n_components=52, random_state=2016)
tfidf_tsvd1 =tsvd.fit_transform(new_text_tfidf)
for i in range(tfidf_tsvd1.shape[1]):
    TT_data['new_tfidf_tsvd' + str(i)] = tfidf_tsvd1[:,i]

In [63]:
tsvd = TruncatedSVD(n_components=52, random_state=2016)
tfidf_tsvd5 =tsvd.fit_transform(vari_text_tfidf)
for i in range(tfidf_tsvd5.shape[1]):
    TT_data['vari_tfidf_tsvd' + str(i)] = tfidf_tsvd5[:,i]

In [64]:
tsvd = TruncatedSVD(n_components=52, random_state=2016)
tfidf_tsvd7 =tsvd.fit_transform(muts_text_tfidf)
for i in range(tfidf_tsvd7.shape[1]):
    TT_data['muts_tfidf_tsvd' + str(i)] = tfidf_tsvd7[:,i]

In [65]:
tsvd = TruncatedSVD(n_components=52, random_state=2016)
tfidf_tsvd8 =tsvd.fit_transform(cells_text_tfidf)
for i in range(tfidf_tsvd8.shape[1]):
    TT_data['cells_tfidf_tsvd' + str(i)] = tfidf_tsvd8[:,i]

In [66]:
#BOW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english', max_features =1000)


In [67]:
new_text_cv = cv.fit_transform(TT_new_text['text'])

In [68]:
tsvd = TruncatedSVD(n_components=52, random_state=2016)
bow_tsvd = tsvd.fit_transform(new_text_cv)
for i in range(bow_tsvd.shape[1]):
    TT_data['new_bow_tsvd' + str(i)] = bow_tsvd[:,i]

In [71]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = [cosine_similarity(new_text_tfidf[i],vari_text_tfidf[i])[0][0] for i in range(new_text_tfidf.shape[0])]
TT_data['cos_sim'] = cos_sim

In [72]:
TT_data.head()

,Gene_len,Gene_lbl_cnc,Var_len,Var_lbl_cnc,text_len,text_word_count,new_tfidf_tsvd0,new_tfidf_tsvd1,new_tfidf_tsvd2,new_tfidf_tsvd3,...,new_bow_tsvd43,new_bow_tsvd44,new_bow_tsvd45,new_bow_tsvd46,new_bow_tsvd47,new_bow_tsvd48,new_bow_tsvd49,new_bow_tsvd50,new_bow_tsvd51,cos_sim
0,6,447,20,7654,39672,6105,0.545821,0.106699,-0.005733,-0.132707,...,-4.483006,-9.506209,2.343717,-3.961367,-6.361186,-0.895882,-0.855018,-2.984720,0.415536,0.214325
1,3,216,5,8255,36691,5783,0.321223,-0.081788,-0.059409,-0.033962,...,-1.084099,1.579904,1.914404,-2.640575,1.093888,0.081520,-1.521776,-1.347583,-0.671877,0.171627
2,3,216,5,5191,36691,5783,0.321223,-0.081788,-0.059409,-0.033962,...,-1.084099,1.579904,1.914404,-2.640575,1.093888,0.081520,-1.521776,-1.347583,-0.671877,0.155785
3,3,216,5,4572,36238,5625,0.632411,-0.018920,-0.009728,0.024229,...,0.371905,4.228816,18.763676,-20.396324,9.506310,4.371376,-7.526444,-17.522809,9.785788,0.231186
4,3,216,5,3958,41308,6248,0.609788,0.047313,-0.179302,0.069652,...,2.113807,30.442861,11.817908,-22.665281,28.943458,33.333660,-7.465071,-11.551889,-4.041435,0.099988


In [73]:
len(TT_data)

8989

In [74]:
X = TT_data[0:len(train_var)]

In [75]:
X_test = TT_data[len(train_var):8989]
y_test = np.zeros((X_test.shape[0],max(y) +1))

In [76]:
#y_test = np.zeros((X_test.shape[0],max(y) +1))
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import *


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [82]:
import xgboost as xgb
results=[]
denom = 0
fold = 5 #Change to 5, 1 for Kaggle Limits
for i in range(fold):
    params = {
        'eta': 0.045,
        'max_depth': 5,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = model_selection.train_test_split(X, y, test_size=0.18, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    score1 = metrics.log_loss(y2, model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), labels = list(range(9)))
    print(score1)
    
    results.append(score1)
    if denom != 0:
        pred = model.predict(xgb.DMatrix(X_test), ntree_limit=model.best_ntree_limit+80)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(X_test), ntree_limit=model.best_ntree_limit+80)
        preds = pred.copy()
   
    denom += 1

preds /= denom

[0]	train-mlogloss:2.10968	valid-mlogloss:2.12794
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:0.76081	valid-mlogloss:1.12096
[100]	train-mlogloss:0.448131	valid-mlogloss:0.95304
[150]	train-mlogloss:0.306317	valid-mlogloss:0.915755
[200]	train-mlogloss:0.216415	valid-mlogloss:0.906394
[250]	train-mlogloss:0.158754	valid-mlogloss:0.914026
[300]	train-mlogloss:0.122072	valid-mlogloss:0.928661
Stopping. Best iteration:
[206]	train-mlogloss:0.20874	valid-mlogloss:0.905485

0.905485365542
[0]	train-mlogloss:2.11092	valid-mlogloss:2.12495
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:0.75852	valid-mlogloss:1.11784
[100]	train-mlogloss:0.437264	valid-mlogloss:0.95629
[150]	train-mlogloss:0.290224	valid-mlogloss:0.922422
[200]	train-mloglos

In [151]:
submission = pd.DataFrame(preds, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = ID_test
submission.head()
submission.to_csv('output.csv', index=False)